# Домашнее задание по теме «Рекуррентные сети 3»

1. Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
2. Обучите на них aeq2aeq по аналогии с заня4тием. Оцените полученное качество
3. Попробуйте добавить +1 рекуррентный слой в encoder и decoder
4. Попробуйте заменить GRU ячейки на ltsm-чейки6. Оцените качество во всех случаях
5. Добавьте в лекционный ноутбук, в функцию train, обучение батчами

# Загрузка библиотек

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import time
import math

In [7]:
import codecs
import os
from io import open

In [8]:
import unicodedata
import string
import re
import random

In [9]:
import numpy as np
import pandas as pd

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [11]:
from tqdm.notebook import tqdm

In [12]:
from itertools import product

# Параметры GPU

In [14]:
print(f"CUDA поддерживается системой?")
if torch.cuda.is_available() == True:
    print(f"CUDA-версия: {torch.version.cuda}")
    cuda_id = torch.cuda.current_device()
    print(f"ID текущего CUDA устройства:{torch.cuda.current_device()}")
    print(f"Имя текущего CUDA устройства:{torch.cuda.get_device_name(cuda_id)}")
else:
    print(f"Нет")

CUDA поддерживается системой?
CUDA-версия: 11.8
ID текущего CUDA устройства:0
Имя текущего CUDA устройства:NVIDIA GeForce RTX 2060


# Выбор процессора

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Импорт данных

In [18]:
file = codecs.open( "rus.txt", "r", "utf-8" )
data = file.read()
file.close()

In [19]:
print(data[-2000:])

ng.	Поскольку сайтов, посвящённых какой-либо теме, как правило, несколько, я обычно просто нажимаю на кнопку "назад", если попадаю на страницу со всплывающей рекламой. Я просто перехожу на следующую страницу, найденную гуглом, и надеюсь найти что-то менее раздражающее.	CC-BY 2.0 (France) Attribution: tatoeba.org #954270 (CK) & #6383010 (odexed)
If someone who doesn't know your background says that you sound like a native speaker, it means they probably noticed something about your speaking that made them realize you weren't a native speaker. In other words, you don't really sound like a native speaker.	Если кто-то незнакомый говорит, что вы говорите как носитель языка, это значит, что он, вероятно, заметил что-то в вашей речи, что дало ему понять, что вы не носитель. Другими словами, вы не говорите как носитель.	CC-BY 2.0 (France) Attribution: tatoeba.org #953936 (CK) & #10644468 (notenoughsun)
Doubtless there exists in this world precisely the right woman for any given man to marry an

# Входные параметры

In [21]:
SEED = 1234
random.seed(SEED)
MAX_LENGTH = 10
num_file = 1
learn_rate = 0.0001
teach_force_ratio = 0.5
num_iters = 30000
every = num_iters/10
SOS_token = 0
EOS_token = 1
lang1 = 'rus'
lang2 = 'eng'

In [22]:
if device=='cuda':
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
else:
  torch.manual_seed(SEED)

In [23]:
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
rnn_types = [nn.RNN, nn.LSTM, nn.GRU]
num_hidden_lay_list = [1,2]
cols = ['RNN_Type', 'Hidden_layers', 'Loss']
PATH = os.path.abspath("rus.txt")

# Функции

## Фасовка слов по словарям

In [26]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

## Кодировка в ASCII

In [28]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

## Нормализация строк

In [30]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-ЯёЁ.!?]+", r" ", s)
    # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

## Чтение файла

In [32]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('rus.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')][:2] for l in lines] #

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

## Токенизация слов

In [34]:
def filterPair(p, prefixes):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(prefixes)

def filterPairs(pairs, prefixes):
    return [pair for pair in pairs if filterPair(pair, prefixes)]

## Подготовка данных

In [36]:
def prepareData(lang1, lang2, prefixes, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs, prefixes)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

## Кодировщик

In [38]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, rnn_type):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

## Декодировщик

In [40]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers, rnn_type):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn       = rnn_type(hidden_size, hidden_size, n_layers)
        self.out       = nn.Linear(hidden_size, output_size)
        self.softmax   = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output         = self.embedding(input).view(1, 1, -1)
        output         = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output         = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

## Формирование тензора

In [42]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## Обучение

In [44]:
def train(input_tensor, target_tensor,
          encoder, decoder, encoder_optimizer, decoder_optimizer,
          criterion, max_length, rnn_type, n_layers, teacher_forcing_ratio):
    if rnn_type.__name__ == 'LSTM':
        encoder_hidden = (encoder.initHidden(n_layers), encoder.initHidden(n_layers))
    else:
        encoder_hidden = encoder.initHidden(n_layers)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

## Для расчёта длительности обучения

In [46]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## Итеративное обучение

In [48]:
def trainIters(pair_sentenses, encoder, decoder, learning_rate, n_iters,
               max_length, rnn_type, num_layers, teacher_forcing_ratio, input_lan, output_lan,
               print_every=5000, plot_every=500):
    start = time.time()
    print_loss_total_loc = 0
    print_loss_avg = 0

    encoder_optimizer = optim.Adam(encoder.parameters())
    decoder_optimizer = optim.Adam(decoder.parameters())

    training_pairs = [tensorsFromPair(random.choice(pair_sentenses),
                                      input_lan, output_lan)
                                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder,
                     encoder_optimizer, decoder_optimizer, criterion,
                     max_length, rnn_type, num_layers, teacher_forcing_ratio)
        print_loss_total_loc += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total_loc / print_every
            print_loss_total_loc = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
    return print_loss_avg

## Оценка

In [52]:
def evaluate(encoder, decoder, sentence, max_length, num_layers,
             rnn_type, inp_lang, out_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(inp_lang, sentence)
        input_length = input_tensor.size()[0]

        if rnn_type.__name__ == 'LSTM':
            encoder_hidden = (encoder.initHidden(num_layers), encoder.initHidden(num_layers))
        else:
            encoder_hidden = encoder.initHidden(num_layers)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(out_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words


def evaluateRandomly(pair_sen, encoder, decoder, max_length,
                     _lauers, rnn_, in_lang, o_lang,  n=10):
    for i in range(n):
        pair = random.choice(pair_sen)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0], MAX_LENGTH,
                                 _lauers, rnn_, in_lang, o_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Функция для тестирования параметров по п. 3 и 4 задания

In [54]:
def test_model(max_length_sent, num_hidden_lay_list, hidden_size = 256,
               learn_rate = 0.006, prefixes=eng_prefixes, reverse=True,
               teach_force_ratio = 0.5,
               num_iters = 50000, every = 1000, num_file = 998):
    
        encoder2 = EncoderRNN(input_lang.n_words, hidden_size, num_layers, rnn_type
                               ).to(device)
        decoder2 = DecoderRNN(hidden_size, output_lang.n_words, num_layers, rnn_type
                               ).to(device)

        print(f'{rnn_type.__name__}, {num_layers} layers\n\ntraining')
        print('===========================')

        loss_ = trainIters(pair_s, encoder2, decoder2, learn_rate, num_iters,
                          max_length_sent, rnn_type, num_layers, teach_force_ratio,
                          input_lang, output_lang, every)

        print('\nevaluation\n')

        evaluateRandomly(pair_s, encoder2, decoder2, max_length_sent,
                         num_layers, rnn_type, input_lang, output_lang)
        print('----------------------------------------------------------------------')
        string = [rnn_type.__name__, num_layers, loss_]
        df.loc[len(df)] = string

# Обучение

## Подготовительное

In [57]:
input_lang, output_lang, pair_s = prepareData(lang1, lang2, prefixes=eng_prefixes, reverse=True)
print(random.choice(pair_s))
df = pd.DataFrame(columns=cols)

Reading lines...
Read 496059 sentence pairs
Trimmed to 28719 sentence pairs
Counting words...
Counted words:
eng 10177
rus 4303
['я младше его на два года .', 'i m two years younger than he is .']


## Тестирование параметров из задания

In [59]:
num_file = 1
for rnn_type, num_layers in product(rnn_types, num_hidden_lay_list):
  test_model(max_length_sent = MAX_LENGTH, num_hidden_lay_list = num_hidden_lay_list,
           learn_rate = learn_rate, prefixes=eng_prefixes, reverse=True, teach_force_ratio = teach_force_ratio,
           num_iters = num_iters, every = every, num_file = num_file)
  num_file += 1

RNN, 1 layers

training


  0%|          | 0/30000 [00:00<?, ?it/s]

1m 3s (- 9m 33s) (3000 10%) 3.4043
2m 13s (- 8m 55s) (6000 20%) 3.2492
3m 26s (- 8m 2s) (9000 30%) 3.2702
4m 37s (- 6m 56s) (12000 40%) 3.2185
5m 48s (- 5m 48s) (15000 50%) 3.1966
7m 8s (- 4m 45s) (18000 60%) 3.1657
8m 24s (- 3m 36s) (21000 70%) 3.2519
9m 38s (- 2m 24s) (24000 80%) 3.2258
10m 42s (- 1m 11s) (27000 90%) 3.1517
11m 49s (- 0m 0s) (30000 100%) 3.1952

evaluation

> я не сержусь на тома .
= i m not mad at tom .
< i m not sure . <EOS>

> я уверен что том сказал правду .
= i m sure that tom told the truth .
< i m not sure . <EOS>

> она сказала что не очень счастлива .
= she said that she wasn t very happy .
< she is into a hat . <EOS>

> ты сегодня вечером очень разговорчив .
= you re very talkative tonight .
< you re not only . <EOS>

> вы все такая же .
= you re still the same .
< you re not only . <EOS>

> вы прекрасно подходите для этои работы .
= you re perfect for the job .
< you re not only . <EOS>

> я слишком хочу спать чтобы делать домашнюю работу .
= i m too sleep

  0%|          | 0/30000 [00:00<?, ?it/s]

1m 20s (- 12m 7s) (3000 10%) 3.3902
2m 29s (- 9m 58s) (6000 20%) 3.2538
3m 46s (- 8m 47s) (9000 30%) 3.1834
4m 58s (- 7m 28s) (12000 40%) 3.1201
6m 25s (- 6m 25s) (15000 50%) 3.1506
7m 51s (- 5m 14s) (18000 60%) 3.1164
9m 26s (- 4m 2s) (21000 70%) 3.0664
11m 0s (- 2m 45s) (24000 80%) 3.1055
12m 30s (- 1m 23s) (27000 90%) 3.1136
14m 9s (- 0m 0s) (30000 100%) 3.0974

evaluation

> они не боятся тяжелои работы .
= they re not afraid of hard work .
< they re not to . . . <EOS>

> я в исподнем .
= i m in my underwear .
< i m glad you re to help . <EOS>

> извини я должен идти .
= i m sorry i have to go .
< i m sorry i was that . <EOS>

> за мнои следят .
= i m being watched .
< you re not going to . . <EOS>

> ты чудовище .
= you re a monster .
< you re very attentive . <EOS>

> она бросила сердитыи взгляд на грубого торговца .
= she scowled at the rude salesman .
< she is an . . . . <EOS>

> я впечатлен вашеи работои .
= i m impressed with your work .
< i m glad you re to help . <EOS>

> о

...

1m 7s (- 10m 3s) (3000 10%) 3.1206
2m 10s (- 8m 42s) (6000 20%) 2.7569
3m 14s (- 7m 33s) (9000 30%) 2.5595
4m 18s (- 6m 27s) (12000 40%) 2.4280
5m 22s (- 5m 22s) (15000 50%) 2.3282
6m 26s (- 4m 17s) (18000 60%) 2.2626
7m 29s (- 3m 12s) (21000 70%) 2.1437
8m 33s (- 2m 8s) (24000 80%) 2.0417
9m 41s (- 1m 4s) (27000 90%) 2.0277
10m 45s (- 0m 0s) (30000 100%) 1.9778

evaluation

> я очень рад что это случилось .
= i m very happy that it happened .
< i m really glad i did . <EOS>

> они все очень голодны .
= they are all very hungry .
< they re all very hungry . <EOS>

> ты очень важен для меня .
= you re very important to me .
< you re very lot me me . <EOS>

> это я написал этот рассказ .
= i m the one who wrote this story .
< i m the one who stole this eggs . <EOS>

> мы к этому не готовы .
= we re not ready for this .
< we re not afraid of that . <EOS>

> я так счастлив что ты вернулся .
= i m so happy you re back .
< i m so glad you re back . <EOS>

> ты не уволена .
= you re not fired

  0%|          | 0/30000 [00:00<?, ?it/s]

0m 48s (- 7m 13s) (3000 10%) 3.0362
1m 32s (- 6m 11s) (6000 20%) 2.6074
2m 18s (- 5m 22s) (9000 30%) 2.3112
3m 3s (- 4m 34s) (12000 40%) 2.1636
3m 47s (- 3m 47s) (15000 50%) 2.0573
4m 32s (- 3m 1s) (18000 60%) 1.9107
5m 17s (- 2m 16s) (21000 70%) 1.8403
6m 2s (- 1m 30s) (24000 80%) 1.7590
6m 47s (- 0m 45s) (27000 90%) 1.7161
7m 32s (- 0m 0s) (30000 100%) 1.6392

evaluation

> я уверен что том не трус .
= i m sure tom isn t a coward .
< i m sure tom tom t that . <EOS>

> вы не дурак .
= you aren t stupid .
< you re not stupid . <EOS>

> я слишком маленькии чтобы идти в школу .
= i m too young to go to school .
< i m too young to go school . <EOS>

> еи нужна помощь .
= she is in need of help .
< we re running out of help . <EOS>

> он идеальныи муж для меня .
= he is an ideal husband for me .
< he is more me me . <EOS>

> я собираюсь одобрить твои план .
= i m going to approve your plan .
< i m going to plan your plan . <EOS>

> мы устраиваем тому вечеринку на день рождения .
= we re th

  0%|          | 0/30000 [00:00<?, ?it/s]

2m 36s (- 23m 26s) (3000 10%) 3.1042
5m 43s (- 22m 52s) (6000 20%) 2.7018
8m 32s (- 19m 55s) (9000 30%) 2.5099
10m 55s (- 16m 22s) (12000 40%) 2.3096
13m 28s (- 13m 28s) (15000 50%) 2.2131
16m 15s (- 10m 50s) (18000 60%) 2.1372
19m 4s (- 8m 10s) (21000 70%) 1.9537
21m 42s (- 5m 25s) (24000 80%) 1.9771
24m 31s (- 2m 43s) (27000 90%) 1.8822
27m 15s (- 0m 0s) (30000 100%) 1.8572

evaluation

> я устал слушать твои жалобы .
= i m tired of listening to your complaints .
< i m tired of your complaints . <EOS>

> я твоего разрешения не спрашиваю .
= i m not asking your permission .
< i m not strong . <EOS>

> она расчесывается .
= she is brushing her hair .
< she s a . . <EOS>

> нам не терпится начать .
= we re looking forward to getting started .
< we re not happy . <EOS>

> мне платят достаточно .
= i m paid enough .
< i m going to need . <EOS>

> он не берет трубку .
= he isn t answering his phone .
< he s not very good . <EOS>

> я мою посуду .
= i m doing the dishes .
< i m a . . <EOS>


# Итоги

In [61]:
df.sort_values(by='Loss')

,RNN_Type,Hidden_layers,Loss
4,GRU,1,1.639242
2,LSTM,1,1.852176
5,GRU,2,1.857226
3,LSTM,2,1.977753
1,RNN,2,3.097398
0,RNN,1,3.195249


Лучшее качество модели по loss показала модель вида GRU с 1 рекуррентным слоём.